In [13]:
using Distributions, StatsPlots, StatsFuns,Plots, Interact, WebIO, LinearAlgebra

# Homework 3

## Consider the bivariate normal distribution we introduced in the class. Let's make the assumptions that  $\mu_x = 0$, $\mu_y = \mu_y$, $\sigma_x = \sigma_x$, $\sigma_y = 1$, and $\rho=\rho$.

### Write down the conditional distributions of X and Y. Note that both of them would be normal distributions. (See Section 1.4.3 and 1.4.3.1 of the lecture note.)

In [25]:
function X_condition_on_y(y, mu_y, sigma_x, rho) 
    return Normal(rho*sigma_x*(y-mu_y), sigma_x*sqrt(1-rho^2))
end

function Y_condition_on_x(x, mu_y, sigma_x, rho) 
    return Normal(mu_y+rho*(1/sigma_x)*(x), sqrt(1-rho^2))
end

Y_condition_on_x (generic function with 1 method)

### Write a Julia function of Gibbs sampler that allows users to choose: the number of sampling points $N$, the values of $\mu_y$, $\sigma_x$, $\rho$, and initial values of $(x_0, y_0)$, and the number of burn-in points. You should follow the steps in Section 1.3 of the lecture note to draw $(x_n, y_n), n=1,\ldots,N$.


In [26]:
struct pair
    x
    y
end

In [27]:
function Gibbs_BN(N, mu_y, sigma_x, rho, init=[2,2], burn_in=10) # total length: 1+skip+N; "1" for initial value
    x = ones(1+burn_in+N)*init[1]
    y = ones(1+burn_in+N)*init[2]
    
    for n = 2:burn_in+N+1
        x[n] = rand(X_condition_on_y(y[n-1], mu_y, sigma_x, rho)) 
        y[n] = rand(Y_condition_on_x(x[n], mu_y, sigma_x, rho)) 
    end
    return pair(x[1+burn_in+1:end], y[1+burn_in+1:end])
end



Gibbs_BN (generic function with 3 methods)

In [28]:
data_x=Gibbs_BN(1000, 1, 2, 0.5, [2,2], 100).x
data_y=Gibbs_BN(1000, 1, 2, 0.5, [2,2], 100).y

1000-element Vector{Float64}:
  2.3439568494306395
  1.8932316672299283
  1.5904055689716436
  1.1495238458474542
 -0.005863154275664861
  1.1864983017464694
 -0.187997367698979
  0.4960562514732324
  0.9750293667585088
  1.359257342908533
  1.3498561963120717
  2.3559486105777245
  1.6255725040565903
  ⋮
  1.8663234499928063
  0.8802926383509659
  0.9670419125642277
  1.524598635268812
  2.7044276186884324
  0.5731075192644759
  0.8707357995175523
  1.9879827195647517
  0.8308174883158416
  0.06488865610777828
  0.6616265868733069
  2.04000080617863

### Draw a graph  similar to the one we showed in the class. Remember to add sliders for both $N$ and $\rho$.

In [17]:
print("Input the sigma_x: ")
sigma_x = parse(Float64, readline())
print("Input the mu_y: ")
mu_y = parse(Float64, readline())
print("Input the max sampling number: ")
Num = parse(Int64, readline())
σ=[sigma_x 0;
    0 1]
μ=[1,mu_y]
draw0 = [rand(Normal(0,sigma_x),Num)';rand(Normal(mu_y,1),Num)']
draw0

Input the sigma_x: stdin> 5
Input the mu_y: stdin> 1
Input the max sampling number: stdin> 5000


2×5000 Matrix{Float64}:
 -6.76732  -0.78446   -7.52183  -1.83902   …  -1.28844   1.31923  -2.72954
  2.21572   0.329976   2.26661  -0.350576      0.542925  1.83861   0.339691

In [19]:
@manipulate for N = 200:50:Num, ρ=-1:0.1:1
    correlation = [1 ρ
                   ρ 1]
    covariance = σ*correlation*σ
    C=cholesky(covariance)
    draw = C.L * draw0[:,1:N] .+μ

    x, y =  draw[1,:] , draw[2,:] 

    layout = @layout [a            _
                      b{0.8w,0.8h} c]

    default(fillcolor = :grey, markercolor = :white, grid = false, legend = false)
    plot(layout = layout, link = :both, size = (400, 400), margin = -10Plots.px)
    scatter!(x,y, subplot = 2, framestyle = :box)
    histogram!([x y], subplot = [1 3], orientation = [:v :h], bins=100 , framestyle = :none)
end

Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Scope(Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :label), Any["N"], Dict{Symbol, Any}(:className => "interact ", :style => Dict{Any, Any}(:padding => "5px 10px 0px 10px")))], Dict{Symbol, Any}(:className => "interact-flex-row-left")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :input), Any[], Dict{Symbol, Any}(:max => 97, :min => 1, :attributes => Dict{Any, Any}(:type => "range", Symbol("data-bind") => "numericValue: index, valueUpdate: 'input', event: {change: function (){this.changes(this.changes()+1)}}", "orient" => "horizontal"), :step => 1, :className => "slider slider is-fullwidth", :style => Dict{Any, Any}()))], Dict{Symbol, Any}(:className => "interact-flex-row-center")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :p), Any[], Dict{Symbol, Any}(:attributes => Dict("data-bind" => "text: formatted_val")))], Dict{Symbol, Any}(:className => "interact-flex-row-right"))], Dict{Symbol, Any}(:className => "interact-flex-row interact-widget")), Dict{String, Tuple{Observables.AbstractObservable, Union{Nothing, Bool}}}("changes" => (Observable{Int64} with 1 listeners. Value:
0, nothing), "index" => (Observable{Any} with 2 listeners. Value:
49, nothing)), Set{String}(), nothing, Asset[Asset("js", "knockout", "C:\\Users\\User\\.julia\\packages\\Knockout\\HReiN\\src\\..\\assets\\knockout.js"), Asset("js", "knockout_punches", "C:\\Users\\User\\.julia\\packages\\Knockout\\HReiN\\src\\..\\assets\\knockout_punches.js"), Asset("js", nothing, "C:\\Users\\User\\.julia\\packages\\InteractBase\\Qhvxg\\src\\..\\assets\\all.js"), Asset("css", nothing, "C:\\Users\\User\\.julia\\packages\\InteractBase\\Qhvxg\\src\\..\\assets\\style.css"), Asset("css", nothing, "C:\\Users\\User\\.julia\\packages\\Interact\\PENUy\\src\\..\\assets\\bulma_confined.min.css")], Dict{Any, Any}("changes" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"changes\"]()) ? (this.valueFromJulia[\"changes\"]=true, this.model[\"changes\"](val)) : undefined})")], "index" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"index\"]()) ? (this.valueFromJulia[\"index\"]=true, this.model[\"index\"](val)) : undefined})")]), WebIO.ConnectionPool(Channel{Any}(32), Set{AbstractConnection}(), Condition(Base.InvasiveLinkedList{Task}(Task (runnable) @0x0000000058782bd0, Task (runnable) @0x0000000058782bd0), Base.AlwaysLockedST(1))), WebIO.JSString[WebIO.JSString("function () {\n    var handler = (function (ko, koPunches) {\n    ko.punches.enableAll();\n    ko.bindingHandlers.numericValue = {\n        init: function(element, valueAccessor, allBindings, data, context) {\n            var stringified = ko.observable(ko.unwrap(valueAccessor()));\n            stringified.subscribe(function(value) {\n                var val = parseFloat(value);\n                if (!isNaN(val)) {\n                    valueAccessor()(val);\n                }\n            });\n            valueAccessor().subscribe(function(value) {\n                var str = JSON.stringify(value);\n                if ((str == \"0\") && ([\"-0\", \"-0.\"].indexOf(stringified()) >= 0))\n                     return;\n                 if ([\"null\", \"\"].indexOf(str) >= 0)\n                     return;\n                stringified(str);\n            });\n            ko.applyBindingsToNode(\n                element,\n                {\n                    value: stringified,\n                    valueUpdate: allBindings.get('valueUpdate'),\n                },\n                context,\n            );\n        }\n    };\n    var json_data = {\"formatted_vals\":[\"200\",\"250\",\"300\",\"350\",\"400\",\"450\",\"500\",\"550\",\"600\",\"650\",\"700\",\"750\",\"800\",\"850\",\"900\",\"950\",\"1000\",\"1050\",\"1100\",\"1150\",\"1200\",\"1250\",\"1300\",\"1350\",\"1400\",\"1450\",\"15